# User interface

In [ ]:
# Creates interface to schedule and run preprocessing jobs
# Define the dictionary and the display functions
from nilearn.plotting import plot_anat, show
import ipywidgets as widgets
import utils
import preprocess_functions
import os
from importlib import reload
from nilearn import plotting
from nilearn import image as nli
reload(utils)
reload(preprocess_functions)
import nibabel as nib
import numpy as np
from ipywidgets import HBox, VBox
import numpy as np
from nilearn.plotting import plot_anat, show
import ipywidgets as widgets
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import ipywidgets as widgets
from IPython.display import display, Markdown, clear_output
import pandas as pd

reload(utils)
reload(preprocess_functions)

run_prepro = False # run full preprocessing?
combination_options = [['x','y','z'],['-x','z','y'],['x','z','-y'],['-x','y','z']]
project_dict = {
    "User": "lauveri",
    "Dataset": "EmoB",
    "Session": "",
    "Task": "EmoB",
    "Participants": [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19],
    "Runs": [1,2,3,4],
    "Sessions": ["01", "02", "03"],
    "Specie": "D",
    "Atlas_type": "Nitzsche",
    "Combination": combination_options[3],
    "Variation": "STD0" # Versions of flirt for normalization STD0 (-90 to 90, default) STD1 (-30 to 30))
}

process_dict = {
    'preprocess_run': {'name': 'Preprocess', 'by_run': True, 'process_used': 'preprocess_run'},
    'get_mean_fct': {'name': 'Mean fct', 'by_run': False, 'process_used': 'get_mean_fct'},
    'mean_to_STD': {'name': 'Mean to atlas', 'by_run': False, 'process_used': 'mean_to_STD'},
    'run_to_STD': {'name': 'Runs to atlas', 'by_run': True, 'process_used': 'run_to_STD'},
    'process_motion': {'name': 'Motion', 'by_run': False, 'process_used': 'process_motion'},
}

process_name_list = [process[1]['name'] for process in process_dict.items()]



if project_dict['User']:
    username = project_dict['User']
    if os.name == 'nt':
        datafolder = r"P:\userdata" + os.sep + username + r"\data"
    # if it is any macOS
    elif os.name == 'posix':
        datafolder = '/Users'+ os.sep + username + os.sep + 'data'
    else:
        datafolder = '/home' + os.sep + username + '/mnt/a471/userdata/' + username + '/data'
else:
    datafolder = ''
# assign to project dict
project_dict['Datafolder'] = datafolder

# Create job_dict to store the process information
job_dict = {
    "User":        project_dict['User'],
    "Dataset":     project_dict['Dataset'],
    "Session":     project_dict['Session'],
    "Task":        project_dict['Task'],
    "sub_N":       project_dict['Participants'][0], # based on sub_selector
    "run_N":  project_dict['Runs'], # based on project_dict
    "Sessions":  project_dict['Sessions'],
    "Specie":      project_dict['Specie'],
    "Process":     process_name_list[0], # based on process_selector
    "Datafolder":  project_dict['Datafolder'],
    "Status":      "Scheduled",
    "Atlas_type":  project_dict['Atlas_type'],
    "Combination": project_dict['Combination'],
    "Smooth":      0,
    "Full_prepro": run_prepro,
    "Variation":   project_dict['Variation'],
    "session_and_run": [], # based on run_selector
}

# Create a list to store the jobs
job_list = [] 

### Input widgets ###

# Create a tab widget
tab = widgets.Tab()

# Create an output widget to display the output of the functions
output = widgets.Output()

# Create the sub_selector dropdown for participants
sub_selector = widgets.Dropdown(
    options=[f"{project_dict['Specie']}-sub-{i:02d}" for i in project_dict['Participants']],
    value=f"{project_dict['Specie']}-sub-{project_dict['Participants'][0]:02d}",
    description='Participants:',
)

# Create the process_selector ToggleButtons for process based on process_list
process_selector = widgets.ToggleButtons(
    options=[process[1]['name'] for process in process_dict.items()],
    description='Process:',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    # icons=['check'] * 3
)

# Create checkbox to select the runs to process
run_selector = VBox([
    HBox([widgets.Checkbox(value=True, description=f"Run {i} (Session {s})")
          for i in project_dict['Runs']])
    for s in project_dict['Sessions']
])

# create combination dropdown
combination_selector = widgets.Dropdown(value=project_dict['Combination'], options=combination_options, description='Combination:')

# create an empty base_run_selector dropdown to use as template for get_mean_fct based on job_dict['session_and_run]
base_run_selector = widgets.Dropdown(
    options=['1'],
    value='1',
    description='Base run:',
    disabled=False,
)

# create checkbox for prepro_selector
prepro_selector = widgets.Checkbox(value=run_prepro, description="full preprocess?")

# Button to schedule the process, make the button black with white text
run_button = widgets.Button(
    description='Add job',
    disabled=False,
    button_style='success', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Add selected process to the job list   ',
    icon='book',
)

run_jobs_button = widgets.Button(
    description='Run jobs',
    disabled=False,
    button_style='info', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Run the selected jobs',
    icon='mail-forward'
)

## Prepare schedule_table ##

# create empty schedule_table
schedule_table = pd.DataFrame()
html_table = schedule_table.to_html(classes='table table-striped')
table_widget = widgets.HTML(value=html_table)

### Update functions ###

def update_sub_selector(b):
    global sub_selector
    new_options = [f"{project_dict['Specie']}-sub-{i:02d}" for i in project_dict['Participants']]
    sub_selector.options = new_options
    sub_selector.value = new_options[0]

def update_run_selection(b):
    global run_selector
    new_run_selector = VBox(
        [HBox([widgets.Checkbox(value=True, description=f"Run {i} (Session {s})") for i in project_dict['Runs']]) for s in project_dict['Sessions']]
    )
    run_selector = new_run_selector

def update_base_run_selector(b):
    global base_run_selector
    # update run_selector
    # update_run_selection(b)
    # update_job_dict)
    update_job_dict(b)
    # get the session_and_run from job_dict
    session_and_run = job_dict['session_and_run']
    # update the base_run_selector options based on session_and_run
    base_run_selector.options = session_and_run
    # set the value to the first option
    if len(session_and_run) > 0:
        base_run_selector.value = session_and_run[0]
    else:
        base_run_selector.value = ''
    
    

def update_prepro_selection(b):
    global job_dict
    # new_prepro_selector = widgets.HBox(
    #     widgets.Checkbox(value=True, description="full preprocess?")
    # )
    #prepro_selector = new_prepro_selector
    update_job_dict(b)

def update_combination(b):
    global job_dict
    update_job_dict(b)

def update_sub(b):
    global job_dict
    update_job_dict(b)

def update_run_selection_ticks(b):
    global run_selector, job_dict
    # update the run_selector ticks based on existing file status
    with output: # shows the output of the function in the output widget
        for s_idx, session in enumerate(project_dict['Sessions']):
            for i_idx, run in enumerate(project_dict['Runs']):
                checkbox = run_selector.children[s_idx].children[i_idx]
                checkbox.value = preprocess_functions.check_file_status(
                    project_dict,
                    job_dict['sub_N'],
                    run,
                    session,
                    process='get_mean_fct'
                )

    

def update_runs(b):
    global job_dict
    update_job_dict(b)
    # update_run_selection_ticks(b)
    
def update_job_dict(b):
    global job_dict
    job_dict['run_N'] = [int(run.description.split()[1]) for session in run_selector.children for run in session.children if run.value]
    job_dict['Sessions'] = [run.description.split()[-1].strip('()') for session in run_selector.children for run in session.children if run.value]
    # add session_and_run in format ['ses-01_run-01','ses-01_run-02'] based on run_selector
    session_and_run = []
    for session in run_selector.children:
        for run in session.children:
            if run.value:
                # run.description == "Run 3 (Session 2)"
                parts = run.description.split()                     # ["Run","3","(Session","2)"]
                run_num = parts[1]                                  # "3"
                sess_num = parts[-1].strip("()")                    # "2"
                session_and_run.append(f"ses-{sess_num}_run-{run_num}")
                # session_and_run.append(f"ses-{session.description.split()[-1].strip('()')}_run-{run.description.split()[1]}")
    job_dict['session_and_run'] = session_and_run
    job_dict['Process'] = process_selector.value
    job_dict['sub_N'] = int(sub_selector.value.split('-')[-1])
    job_dict['Combination'] = combination_selector.value
    job_dict['Full_prepro'] = prepro_selector.value


def process_selector_button_pressed(b):
    global job_dict, prepro_selector
    # get the key of the process selected matching process_selector.value with process_name_list
    # process_key = [key for key, value in process_dict.items() if value['name'] == job_dict['Process']][0]
    process_key = [key for key, value in process_dict.items() if value['name'] == process_selector.value][0]
    
   
        # prepro_selector.disabled = True
    # Update job_dict 
    update_job_dict(b)
    
   

    # update run_selector ticks based on existing file status
    with output: # shows the output of the function in the output widget
        for s_idx, session in enumerate(project_dict['Sessions']):
            for i_idx, run in enumerate(project_dict['Runs']):
                checkbox = run_selector.children[s_idx].children[i_idx]
                checkbox.value = preprocess_functions.check_file_status(
                    project_dict,
                    job_dict['sub_N'],
                    run,
                    session,
                    process_key
                )
    if process_key == 'preprocess_run':
        prepro_selector.disabled = False
        base_run_selector.disabled = True
    elif process_key == 'get_mean_fct':
        update_base_run_selector(b)
        base_run_selector.disabled = False
    else:
        base_run_selector.disabled = True
### Function to run the jobs ###

# Create function for run_button
def add_job_button_pressed(b):
    global job_dict, job_list, schedule_table
    # update job_dict with the current values of the widgets
    update_job_dict(b)
    # add the job_dict to the job_list
    job_list.append(job_dict.copy())
    #print('job added number of jobs: ' + str(len(job_list)))
    # update the schedule_table
    schedule_table = utils.job_list_to_table(job_list)
    html_table = schedule_table.to_html(classes='table table-striped')
    # update the table_widget
    table_widget.value = html_table


def run_jobs(b):
    global job_list, schedule_table
    with output:
        output.clear_output()
        for n, job in enumerate(job_list):
            print(job)
            if job_list[n]['Status'] == 'Finished':
                continue
            else:
                preprocess_functions.run_process(job)
                job_status = preprocess_functions.check_job_status(job)
                job_list[n]['Status'] = job_status


### Attach functions to widgets ###

# add function to sub_selector
sub_selector.observe(update_sub, names='value')

# add function to process_selector
process_selector.observe(process_selector_button_pressed, names='value')

# add function to run_selector
for session in run_selector.children:
    for run in session.children:
        run.observe(update_runs, names='value')

# add function to prepro_selector
prepro_selector.observe(update_prepro_selection, names='value')

# add function to combination_selector
combination_selector.observe(update_combination, names='value')

# attach function to run_button
run_button.on_click(add_job_button_pressed)

# attach function to run_jobs_button
run_jobs_button.on_click(run_jobs)

## Display the widgets ##

# Create Scheduler tab
scheduler_tab = widgets.VBox([
    sub_selector,
    process_selector,
    prepro_selector,
    combination_selector,
    base_run_selector,
    run_selector,
    run_button,
    run_jobs_button,
    table_widget,
    output,  # this shows your print output
])


# Adding the tabs
tab_list = [scheduler_tab
             ]

tab.children = tab_list
# Set the tab titles
tab_contents = ['Scheduler']
for i, name in enumerate(tab_contents):
    tab.set_title(i, name)



# Display the tab widget
display(tab)

TraitError: Invalid selection: value not found

# Crop app

In [12]:
sub_N = 2
tab_crop_app,out = preprocess_functions.crop_app(project_dict, sub_N)
display(tab_crop_app,out)

Output()

# BET app

In [41]:
sub_N = 1

initial_params = {'bety_1':9,
              'betz_1':13,
              'thr_1':0.65,
              'bety_2':14,
              'betz_2':28,
              'thr_2':0.6,
              'bety_3':25,
              'betz_3':19,
              'thr_3':0.75,
              }

tab_crop_app,out = preprocess_functions.bet_app(project_dict, sub_N, initial_params)
display(tab_crop_app,out)

Output()

# Get vectors and motion

In [52]:
import pandas as pd

def save_fsl_vector(vector_df: pd.DataFrame, filename: str):
    """
    Save a DataFrame with columns 'onset', 'duration', 'one' to a text file
    suitable for FSL, rounding onset to two decimal places and excluding headers.

    Parameters:
    - vector_df (pd.DataFrame): DataFrame with columns ['onset', 'duration', 'one']
    - filename (str): Path to the output text file
    """
    # Round the 'onset' column to two decimals
    df_to_save = vector_df.copy()
    df_to_save['onset'] = df_to_save['onset'].round(2)

    # Save to txt without headers or index
    df_to_save.to_csv(filename, sep='\t', index=False, header=False, float_format='%.2f')


In [ ]:
def concatenate_event_files(file_paths, output_file):
    """
    Concatenates multiple event files with the same structure into one output file.

    Parameters:
    - file_paths (list of str): List of file paths to be concatenated.
    - output_file (str): Path to the output file where content will be written.
    """
    with open(output_file, 'w') as out_f:
        for path in file_paths:
            with open(path, 'r') as f:
                content = f.read()
                out_f.write(content)
                if not content.endswith('\n'):
                    out_f.write('\n')  # Ensure newline between files


In [ ]:
from nilearn.plotting import plot_anat, show
import ipywidgets as widgets
import utils
import preprocess_functions
import os
from importlib import reload
from nilearn import plotting
from nilearn import image as nli
reload(utils)
reload(preprocess_functions)
import nibabel as nib
import numpy as np
from ipywidgets import HBox, VBox
import numpy as np
from nilearn.plotting import plot_anat, show
import ipywidgets as widgets
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import ipywidgets as widgets
from IPython.display import display, Markdown, clear_output
import pandas as pd

import os
import utils
import shutil
reload(utils)
reload(preprocess_functions)

project_dict = {
    "User": "raulh87",
    "Dataset": "EmoB",
    "Session": "",
    "Task": "EmoB",
    "Participants": [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19],
    "Runs": [1,2,3,4],
    "Sessions": ["01", "02", "03"],
    "Specie": "D",
    "Atlas_type": "Nitzsche",
}


if os.name == 'nt':
    datafolder = r"P:\userdata" + os.sep + project_dict['User'] + r"\data"
    # datafolder = r"C:\data"
# if it is any macOS
elif os.name == 'posix':
    datafolder = '/Users'+ os.sep + project_dict['User'] + os.sep + 'data'
else:
    datafolder = '/home' + os.sep + project_dict['User'] + '/mnt/a471/userdata/' + project_dict['User'] + '/data'

# assign to project dict
project_dict['Datafolder'] = datafolder

specie = project_dict['Specie']
dataset = project_dict['Dataset']
task = project_dict['Task']

sub_N = 1
session = 1
run_N = 1

movement_file_path_orig = (datafolder + os.sep + dataset + os.sep + 'preprocessing' + os.sep + 
                      specie + '-sub-' + str(sub_N).zfill(2) + os.sep +
                      specie + '-sub-' + str(sub_N).zfill(2) + 
                      '_ses-' + str(session).zfill(2) + '_task-' + 
                      task + '_run-' + str(run_N).zfill(2) +  '.feat' + os.sep +
                    'mc' + os.sep + 'prefiltered_func_data_mcf.par')
movement_file_path = (datafolder + os.sep + dataset + os.sep + 'movement' + os.sep +
                      specie + '-sub-' + str(sub_N).zfill(2) + 
                      '_ses-' + str(session).zfill(2) + '_task-' + 
                      task + '_run-' + str(run_N).zfill(2) + '.par')

movement_file_txt = (datafolder + os.sep + dataset + os.sep + 'movement' + os.sep +
                      specie + '-sub-' + str(sub_N).zfill(2) +  
                      '_ses-' + str(session).zfill(2) + '_task-' +
                      task + '_run-' + str(run_N).zfill(2) + '_fwd.txt')

print('file: ' + movement_file_path_orig + ' copied to ' + movement_file_path)
# shutil.copyfile(movement_file_path_orig, movement_file_path)

radius=55.0
threshold=0.5
print('Getting framewise displacement')
# pass_fwd = preprocess_functions.fwd(movement_file_path_orig, radius, threshold, movement_file_txt)



input_file = (datafolder + os.sep + dataset + os.sep + 'normalized' + os.sep +
                        specie + '-sub-' + str(sub_N).zfill(2) + os.sep +
                        specie + '-sub-' + str(sub_N).zfill(2) +
                        '_ses-' + str(session).zfill(2) + '_task-' +
                        task + '_run-' + str(run_N).zfill(2) + '.nii.gz')

smooth = 3
atlas_type = project_dict['Atlas_type']
img_type = 'brain2mm'

if specie == 'D':
    specieS = 'Dog'
elif specie == 'H':
    specieS = 'Hum'

# generate path to atlas. The atlas is in the same folder as the script
atlas_file = os.getcwd() + os.sep + "Atlas" + os.sep + specieS + os.sep + atlas_type + os.sep + img_type + ".nii.gz"

TR,volumes = utils.extract_params(input_file)

fsl_outputdir = (datafolder + os.sep + dataset + os.sep + 'results' + os.sep +
                  'GLM' + os.sep + 'model-01' + os.sep +
                    specie + '-sub-' + str(sub_N).zfill(2) + os.sep +
                    'ses-' + str(session).zfill(2) + os.sep +
                    'task-' + task + os.sep +
                    'run-' + str(run_N).zfill(2))
# run-01/cond-01.txt"
condition = (datafolder + os.sep + dataset + os.sep + 'models' + os.sep +
                        'model-01' + os.sep + 'run-' + str(run_N).zfill(2) + os.sep +
                        'cond-01.txt')
                        

label_list = ['outputdir', 'TR', 'volumes', 'BET', 'smooth', 'input', 
              'atlas', 'movement', 'condition']

    # create dictionary to fill in the design.fsf file
to_fill_dict = dict()
for label in label_list:
    to_fill_dict[label] = dict()
    if label == 'outputdir':
        to_fill_dict[label]['string_to_find'] = 'set fmri(outputdir)'
        to_fill_dict[label]['string_to_replace'] = ('set fmri(outputdir) "' + fsl_outputdir + '"')
    elif label == 'TR':
        to_fill_dict[label]['string_to_find'] = 'set fmri(tr)'
        to_fill_dict[label]['string_to_replace'] = ('set fmri(tr) ' + str(TR))
    elif label == 'volumes':
        to_fill_dict[label]['string_to_find'] = 'set fmri(npts)'
        to_fill_dict[label]['string_to_replace'] = ('set fmri(npts) ' + str(volumes))
    elif label == 'BET':
        to_fill_dict[label]['string_to_find'] = 'set fmri(bet_yn)'
        if specie == 'H':
            to_fill_dict[label]['string_to_replace'] = ('set fmri(bet_yn) 1')
        elif specie == 'D':
            to_fill_dict[label]['string_to_replace'] = ('set fmri(bet_yn) 0')
    elif label == 'smooth':
        to_fill_dict[label]['string_to_find'] = 'set fmri(smooth)'
        to_fill_dict[label]['string_to_replace'] = ('set fmri(smooth) ' + str(smooth))
    elif label == 'input':
        to_fill_dict[label]['string_to_find'] = 'set feat_files(1)'
        to_fill_dict[label]['string_to_replace'] = ('set feat_files(1) "' + input_file + '"')
    elif label == 'atlas':
        to_fill_dict[label]['string_to_find'] = 'set fmri(regstandard)'
        to_fill_dict[label]['string_to_replace'] = ('set fmri(regstandard) "' + atlas_file + '"')
    elif label == 'movement':
        to_fill_dict[label]['string_to_find'] = 'set confoundev_files(1)'
        to_fill_dict[label]['string_to_replace'] = ('set confoundev_files(1) "' + movement_file_path + '"')
    elif label == 'condition':
        to_fill_dict[label]['string_to_find'] = 'set fmri(custom1)'
        to_fill_dict[label]['string_to_replace'] = ('set fmri(custom1) "' + condition + '"')
    
design_path = (datafolder + os.sep + dataset + os.sep + 'FSL_designs' + os.sep +   
               'model-01.fsf')
design_modified_path = (datafolder + os.sep + dataset + os.sep + 'FSL_designs' + os.sep +   
               'model-01_modified.fsf')

utils.fill_fsf(to_fill_dict, design_path, design_modified_path)

if os.path.exists(fsl_outputdir + '.feat'):
    shutil.rmtree(fsl_outputdir + '.feat')

command = 'feat ' + design_modified_path

# check if system is windows, if so, do not execute command
print(command)
if os.name == 'nt':
    print("The system is windows, command not executed.")
else:
    os.system(command)


file: P:\userdata\raulh87\data\EmoB\preprocessing\D-sub-01\D-sub-01_ses-01_task-EmoB_run-01.feat\mc\prefiltered_func_data_mcf.par copied to P:\userdata\raulh87\data\EmoB\movement\D-sub-01_ses-01_task-EmoB_run-01.par
Getting framewise displacement
Motion parameters and mask saved to P:\userdata\raulh87\data\EmoB\movement\D-sub-01_ses-01_task-EmoB_run-01_fwd.txt


In [6]:
vector_df

,50.5,4,1
0,78.5,4,1
1,123.5,4,1
2,176.5,4,1
3,231.5,4,1
4,249.5,4,1


# Data quality using FWD and DVARS

In [39]:
# check if the systwm is windows or linux
if os.name == 'nt': # windows
    print('Not importing matlab engine')
    def get_fake_data():
        key_list = ['participant','run_N','fwd', 'fwd_clean', 'volumesLostAfter_fwd', 'totalVolumes', 'dvars', 'dvars_clean', 'volumesLostAfter_dvars', 'lostVolumes_fw_or_dvars', 'lostVolumes_fw_and_dvars']
        table_out = {key: np.random.rand(1) for key in key_list}
        return table_out
        # Create fake data. A dictionary with the keys 
else: # linux
    import matlab.engine
    eng = matlab.engine.start_matlab();
    eng.addpath('Quality_check');

Not importing matlab engine


In [ ]:
user = project_dict['User']
dataset = project_dict['Dataset']
participants = project_dict['Participants']
runs = project_dict['Runs']
specie = project_dict['Specie']
task = project_dict['Task']
session = project_dict['Session']
thr_fwd = 0.5 # threshold for framewise displacement
thr_dvars = 1 # threshold for DVARS
radius = 55.0 # radius for fwd calculation, FSL uses 55.0 as default
whichType = 'fwd' # 'fwd', 'dvars', 'or' or 'and'. See below.
# Takes:
# 'fwd': for generating using only volumes marked by fwd
# 'dvars': for generating using only volumnes marked by dvars
# 'or': will mark the volumes who excede the threshold in any of the two measurements
# 'and': will mark the volumes who excede the threshold in both measurements



# Define the project folder
project_folder = '/home' + os.sep + user + '/mnt/a471/userdata/' + user + '/data' + os.sep + dataset
# Create dataframe to store the output
key_list = ['participant','run_N','fwd', 'fwd_clean', 'volumesLostAfter_fwd', 'totalVolumes', 'dvars', 'dvars_clean', 'volumesLostAfter_dvars', 'lostVolumes_fw_or_dvars', 'lostVolumes_fw_and_dvars']
# create empty quality_table
quality_table = pd.DataFrame(columns=key_list)


for sub_N in participants:
    # indicate the participant
    print(f'Participant {sub_N}')
    for run_N in runs:
        # indicate the run
        print(f'Run {run_N}')
        # check if system is windows or linux
        if os.name == 'nt': # windows
            table_out = get_fake_data()
        else: # linux
            table_out = eng.run_quality_check(project_folder, sub_N, run_N, specie, task, session, thr_fwd, thr_dvars, radius, whichType);
        # Add sub_N and run_N to the table_out
        table_out['participant'] = sub_N
        table_out['run_N'] = run_N
        # Append table_out to quality_table
        quality_table = pd.concat([quality_table, pd.DataFrame([table_out])], ignore_index=True)

# export the quality_table to a csv file
quality_table.to_csv(project_folder + os.sep + 'quality_table.csv')

,participant,run_N,fwd,fwd_clean,volumesLostAfter_fwd,totalVolumes,dvars,dvars_clean,volumesLostAfter_dvars,lostVolumes_fw_or_dvars,lostVolumes_fw_and_dvars
0,1,1,0.904291,0.422286,0.900575,0.383342,0.350866,0.171932,0.707956,0.206182,0.751006
1,1,2,0.123176,0.461794,0.508783,0.694186,0.241390,0.810832,0.126543,0.727711,0.210615
2,1,3,0.662420,0.367324,0.202677,0.846099,0.384206,0.515641,0.721775,0.047864,0.293254
3,2,1,0.544302,0.207957,0.874145,0.998854,0.298997,0.281915,0.340490,0.355109,0.322519
4,2,2,0.942262,0.309705,0.341281,0.791939,0.889615,0.045583,0.918087,0.508759,0.435169
5,2,3,0.608016,0.491991,0.981119,0.917910,0.296719,0.623021,0.800527,0.105881,0.176370
6,3,1,0.781368,0.258295,0.825700,0.353077,0.125040,0.956045,0.141184,0.607822,0.801166
7,3,2,0.794176,0.054675,0.999685,0.375479,0.314170,0.154011,0.714465,0.205141,0.440875
8,3,3,0.691775,0.368913,0.145948,0.168403,0.269429,0.143619,0.262639,0.854616,0.902013
9,4,1,0.154722,0.666698,0.541760,0.413231,0.530316,0.131396,0.196827,0.351643,0.573823


In [13]:
project_dict = {
    "User": "raulh87",
    "Dataset": "Complex",
    "Session": "",
    "Task": "visual",
    "Participants": [1,2,3,4,5],
    "Runs": [1,2,3],
    "Specie": "D",
}
if project_dict['User']:
    username = project_dict['User']
    if os.name == 'nt':
        datafolder = r"P:\userdata" + os.sep + username + r"\data"
    else:
        datafolder = '/home' + os.sep + username + '/mnt/a471/userdata/' + username + '/data'
else:
    datafolder = ''
# assign to project dict
project_dict['Datafolder'] = datafolder
# Create a list to store the jobs
job_list = [] 

# Create the input widgets
input_widgets = create_input_widgets()
update_button = widgets.Button(description="Update")
cancel_button = widgets.Button(description="Cancel")

# Assign an event to the button
update_button.on_click(update_data)
cancel_button.on_click(cancel_update)

# Create a tab widget
tab = widgets.Tab()

# Create the rows to store the widgets
row_1 = widgets.VBox([widgets.VBox(list(input_widgets.values())), update_button, cancel_button])
row_2 = widgets.Output(layout={'border': '1px solid black'})                     

# Create the columns to store in tab 1
data_entry_tab = widgets.HBox([row_1,row_2])

display(data_entry_tab)